In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import plotly.graph_objects as go

from collections import namedtuple

np.random.seed(7)

In [ ]:
ConjugateNormalParams = namedtuple('ConjugateNormalParams', 'mu sigma sx')

def initial_params_normal(mu, sigma, sx):
    return ConjugateNormalParams(mu=mu, sigma=sigma, sx=sx)

def posterior_params_normal(data, initial_pars):
    N = len(data)
    sigma_n_2 = (initial_pars.sigma**2 * initial_pars.sx**2) / (initial_pars.sx**2 + N * initial_pars.sigma**2)
    mu_n = initial_pars.mu * sigma_n_2 / initial_pars.sigma**2 + np.sum(data) * sigma_n_2 / initial_pars.sx**2    
    return ConjugateNormalParams(mu=mu_n, sigma=np.sqrt(sigma_n_2), sx=initial_pars.sx)

def posterior_mu_dist(params):
    return stats.norm(loc=params.mu, scale=params.sigma)

def posterior_rvs(params, nsamp):
    mus = stats.norm.rvs(loc=params.mu, scale=params.sigma, size=nsamp)
    return stats.norm.rvs(loc=mus, scale=params.sx, size=nsamp)

mu = 3
sigma = 1
nsample = 1000
npostsamp = 100000

exact_dist = stats.norm(loc=mu, scale=sigma)
data = exact_dist.rvs(nsample)

sx = np.std(data)
mu0 = data[0]
sigma0 = np.std(data)
pars = initial_params_normal(mu=mu0, sigma=sigma0, sx=sx)
pars = posterior_params_normal(data[1:], pars)
post_mu = posterior_mu_dist(pars)
post_samp = posterior_rvs(pars, npostsamp)

x = np.linspace(0, 10, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=post_mu.pdf(x), line_color='black', name='$\mbox{Апостериорное }\mu$'))
fig.add_trace(go.Scatter(x=[data.mean(), data.mean()], y=[0, max(post_mu.pdf(x))], 
                         line_color='black', mode='lines', line_dash='dash', name='Среднее в выборке'))
fig.add_trace(go.Scatter(x=[exact_dist.mean(), exact_dist.mean()], y=[0, max(post_mu.pdf(x))*1.05], 
                         line_color='red', mode='lines', line_dash='dash', name='Точное среднее'))
fig.update_layout(title='$\mbox{Апостериорное распределение } \mu$',
                  xaxis_title='$\mu$',
                  yaxis_title='Плотность вероятности',
                  xaxis_range=[2, 4],
                  barmode='overlay',
                  hovermode="x",
                  height=500)                  
fig.show()
#fig.write_image("./figs/ch4_norm_postdist_mu.png", scale=2)
#Мода распределения mu близка точному среднему и среднему в выборке.

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), line_color='red', line_dash='dash', name='Точное распределение'))
fig.add_trace(go.Histogram(x=post_samp, histnorm='probability density', name='Сэмпл апостериорного x', nbinsx=100,
                           marker_color='black', opacity=0.8))
fig.update_layout(title='Сэмпл апостериорного распределения x',
                  xaxis_title='x',
                  yaxis_title='Плотность вероятности',
                  #xaxis_range=[0, 10],
                  barmode='overlay',
                  hovermode="x",
                  height=500)                  
fig.show()
#fig.write_image("./figs/ch4_norm_postdist_x.png", scale=2)
#Апостериорное предиктивное распределение x близко исходному нормальному распределению.

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=post_mu.pdf(x), line_color='black', name='$\mbox{Апостериорное }\mu$'))
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), line_color='red', line_dash='dash', name='Точное распределение'))
fig.add_trace(go.Histogram(x=post_samp, histnorm='probability density', name='Сэмпл апостериорного x', nbinsx=100,
                           marker_color='black', opacity=0.8))
fig.update_layout(title='$\mbox{Распределения } x \mbox{ и } \mu$',
                  xaxis_title='x',
                  yaxis_title='Плотность вероятности',
                  xaxis_range=[0, 6],
                  barmode='overlay',
                  hovermode="x",
                  height=500)                  
fig.show()
#fig.write_image("./figs/ch4_norm_postdist_mu_x.png", scale=2)
#Сравнение апостериорных распределений x и mu. Распределение mu - оценка среднего исходного распределения, распределение x - приближение всего исходного нормального распределения. Распределение mu существенно уже.

In [ ]:
def reshape_and_compute_means(sample, n_split):
    n_means = len(sample) // n_split
    samp_reshaped = np.reshape(sample[0 : n_means * n_split], (n_means, n_split))
    means = np.array([x.mean() for x in samp_reshaped])
    return means

def exact_clt_dist(exact_dist, n_split):
    clt_mu = exact_dist.mean()
    clt_stdev = exact_dist.std() / np.sqrt(n_split)
    return stats.norm(loc=clt_mu, scale=clt_stdev)

def sample_clt_dist(means):
    clt_mu = means.mean()
    clt_std = means.std()
    return stats.norm(loc=clt_mu, scale=clt_std)

nsample = 50000
nsplit = 100
npostsamp = 50000

a = 1
b = 2
exact_dist = stats.gamma(a=a, scale=1/b)
data = exact_dist.rvs(nsample)

means = reshape_and_compute_means(data, nsplit)
clt_dist_exact = exact_clt_dist(exact_dist, nsplit)

sx = np.std(means)
mu0 = means[0]
sigma0 = sx
pars = initial_params_normal(mu=mu0, sigma=sigma0, sx=sx)
pars = posterior_params_normal(means[1:], pars)
post_mu = posterior_mu_dist(pars)
post_samp = posterior_rvs(pars, npostsamp)

x = np.linspace(0, 10, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), 
                         mode='lines', line_color='black', line_dash='solid', name='Исходное распределение'))
fig.add_trace(go.Scatter(x=x, y=clt_dist_exact.pdf(x), 
                         mode='lines', line_color='black', line_dash='dash', name='$Norm(\mu, \sigma^2/N)$'))
fig.add_trace(go.Histogram(x=means, histnorm='probability density', name='Выборочные средние', nbinsx=50,
                           marker_color='green', opacity=0.5))
fig.update_layout(title='Выборочные средние',
                  xaxis_title='x',
                  yaxis_title='Плотность вероятности',
                  barmode='overlay',
                  hovermode="x",
                  height=550)
fig.update_layout(xaxis_range=[0, 3])
fig.show()
#fig.write_image("./figs/ch4_gamma_means.png", scale=2)
#Исходное гамма-распределение и выборочные средние. Выборочные средние близки нормальному распределению на основе центральной предельной теоремы.

x = np.linspace(0, 4, 10000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=post_mu.pdf(x), line_color='black', name='$\mbox{Распределение }\mu$'))
fig.add_trace(go.Scatter(x=[data.mean(), data.mean()], y=[0, max(post_mu.pdf(x))], 
                         line_color='black', mode='lines', line_dash='dash', name='Среднее в выборке'))
fig.add_trace(go.Scatter(x=[exact_dist.mean(), exact_dist.mean()], y=[0, max(post_mu.pdf(x))*1.05], 
                         line_color='red', mode='lines', line_dash='dash', name='Точное среднее'))
fig.update_layout(title='$\mbox{Распределение }\mu$',
                  xaxis_title='$\mu$',
                  yaxis_title='Плотность вероятности',
                  xaxis_range=[exact_dist.mean()-0.1, exact_dist.mean()+0.1],
                  barmode='overlay',
                  hovermode="x",
                  height=500)                  
fig.show()
#fig.write_image("./figs/ch4_gamma_postdist_mu.png", scale=2)
#Оценка mu по выборочным средним. Мода распределения близка точному среднему гамма-распределения.


fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), line_dash='solid', line_color='black', name='Исходное распределение'))
fig.add_trace(go.Scatter(x=x, y=clt_dist_exact.pdf(x), 
                         mode='lines', line_color='black', line_dash='dash', name='$Norm(\mu, \sigma^2/N)$'))
fig.add_trace(go.Histogram(x=post_samp, histnorm='probability density', name='$\mbox{Апострериорное } \overline{X}_N$', nbinsx=300,
                           marker_color='black', opacity=0.2))
fig.update_layout(title='$\mbox{Апостериорное распределение } \overline{X}_N$',
                  xaxis_title='x',
                  yaxis_title='Плотность вероятности',
                  xaxis_range=[0, 3],
                  barmode='overlay',
                  hovermode="x",
                  height=500)                  
fig.show()
#fig.write_image("./figs/ch4_gamma_postdist_means.png", scale=2)
#Апостериорное прогнозное распределение выборочных средних близко нормальному на основе центральной предельной теоремы.

In [ ]:
#data.mean()
stderrmean = data.std() / np.sqrt(len(data))
data.mean(), stderrmean

In [ ]:
post_mu.mean(), post_mu.std()